## DNN model for MNIST dataset

In [14]:
import numpy as np
import torch
import tensorflow as tf

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Convert to PyTorch format [batch_size, total pixels]
# Since images are already normalized and flattened for TensorFlow, we can use the same arrays
train_images_pt = torch.tensor(train_images_tf).float()
test_images_pt = torch.tensor(test_images_tf).float()
train_labels_pt = torch.tensor(train_labels)
test_labels_pt = torch.tensor(test_labels)

In [26]:
from tensorflow import keras
num_classes = 10

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
#    keras.layers.Dense(128, activation='relu'),     # Increased number of neurons
    keras.layers.Dense(56, activation='relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [27]:
# Train the model
history = model_tf.fit(train_images_tf, train_labels, epochs=3, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3315 - accuracy: 0.9070 - val_loss: 0.1610 - val_accuracy: 0.9552
Epoch 2/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1630 - accuracy: 0.9522 - val_loss: 0.1183 - val_accuracy: 0.9650
Epoch 3/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1170 - accuracy: 0.9650 - val_loss: 0.1037 - val_accuracy: 0.9700
313/313 - 0s - loss: 0.1174 - accuracy: 0.9647 - 208ms/epoch - 665us/step

Test accuracy: 0.9646999835968018


### Convert to Pytorch DNN model

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        # Fully connected layers / Dense block
        # First dense layer
        self.fc1 = nn.Linear(28*28, 56)  # Flatten 28*28 and feed into 56 neurons
        
        # Second dense layer (output layer)
        self.fc2 = nn.Linear(56, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Flatten the tensor
        #x = x.view(-1, 28*28)
        
        # Fully connected layers with ReLU activation for the first layer
        x = F.relu(self.fc1(x))

        # Output layer with no activation
        # Softmax will be applied externally during training and evaluation
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
model_pt = Net()

In [32]:
# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[1].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [34]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images[36].reshape(1, 28*28)  # Reshape to (1, 784) for DNN
controlled_input_pt = torch.from_numpy(controlled_input_tf).float()

# Test TensorFlow Model
output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)

# Test PyTorch Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)
print("PyTorch Basic Model Output:", torch.exp(output_pt).numpy())

1/1 [==============================] - 0s 8ms/step
TensorFlow Basic Model Output: [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
PyTorch Basic Model Output: [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [36]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset for test data
test_dataset = TensorDataset(test_images_pt, test_labels_pt)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate_pytorch_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_loader)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')


Accuracy of the PyTorch model on the test images: 96.47000000%


In [43]:
def get_predictions_tf(model, test_images, batch_size=32):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=32):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [44]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images_tf)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/1 [==============================] - 0s 14ms/step
Number of mismatches: 0 out of 10000 samples


### Test on Orion


In [45]:
torch.onnx.export(model_pt, controlled_input_pt, "orion_pt_mnist.onnx")

Use Giza-cli to automatically generate cairo script

In [46]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

In [47]:
controlled_input_pt.shape

torch.Size([1, 784])

In [48]:
input_cairo = generate_input_cairo(controlled_input_pt[0])

In [51]:
with open("mnist_cairo/src/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)